
# Part 2. Learning word2vec in pure numpy

## initialisation

In [ ]:
def initialize_parameters(vocab_size, emb_size):
    """
    initialize all the training parameters
    """
    
    E = np.random.normal(loc=0, scale=.01, size=(vocab_size, emb_size))
    W = np.random.normal(loc=0, scale=.01, size=(vocab_size, emb_size))
  
    parameters = {}
    parameters['WRD_EMB'] = E
    parameters['W'] = W
    
    return parameters

In [ ]:
# par exemple un plongement de dimension 5
parameters = initialize_parameters(len(tok2id), 5)
# parameters

In [ ]:
print(parameters['WRD_EMB'].shape, parameters['W'].shape)

In [ ]:
#on peut les multiplier :
(parameters['WRD_EMB'].T @ parameters['W']).shape

## forward propagation

In [ ]:
def forward_propagation(inds, parameters):
    
    m = inds.shape[1]
    WRD_EMB = parameters['WRD_EMB']
    # sélectionner les lignes du plongement correspondant aux indices du batch, transformer -> autant de colonnes que d'indices
    word_vec = WRD_EMB[inds.flatten(), :].T    
    assert(word_vec.shape == (WRD_EMB.shape[1], m))
    
    m = word_vec.shape[1]
    W = parameters['W']
    Z = np.dot(W, word_vec)
    assert(Z.shape == (W.shape[0], m)) 
    
    expZ = np.exp(Z)
    softmax_out = np.divide(expZ, np.sum(expZ, axis=0, keepdims=True) + 0.001)
    assert(softmax_out.shape == Z.shape)
    
    caches = {}
    caches['inds'] = inds
    caches['word_vec'] = word_vec
    caches['W'] = W
    caches['Z'] = Z
    return softmax_out, caches



## cost function

In [ ]:
def cross_entropy(softmax_out, Y):
    """
    softmax_out: output out of softmax. shape: (vocab_size, m=batch_size)
    Y: ground truth: indices à prédire. shape: (1, m)
    """
    m = softmax_out.shape[1]
    
    # todo: comprendre ceci :
    # du softmax out, je sélectionne les lignes correspondantes aux tokens prédits
    # ça me donne les probabilités, j'en prends le log, et j'en fais la moyenne
    # si la prédiction était parfaite, la proba serait 1, le log est 0 --> coût zéro
    # sinon, j'ai une proba plus petite que 1, je fais la moyenne de ces logs de probas (qui sont tous négatifs)
    # avec le "-" ça devient positif, c'est donc une mesure de l'erreur de la prédiction
    # notez que Y peut contenir plusieurs fois le même indice à prédire. Il est comptez autant de fois qu'il devrait être prédit.

    cost = -(1 / m) * np.sum(np.log(softmax_out[Y.flatten(), np.arange(Y.shape[1])] + 0.001))
    return cost


In [ ]:
def softmax_backward(Y, softmax_out):
    """
    Y: labels of training data. shape: (1, m=batch_size)
    softmax_out: output out of softmax. shape: (vocab_size, m=batch_size)
    """
    m = Y.shape[1]
    
    # on calcul la différence entre la prédiction et la ground truth (Y)
    # si la prédiction était parfaite, le dL_dZ devient 0
    # on ne touche qu'aux lignes à prédire
    # si le token n'est pas à prédire, on ne touche pas à la ligne
    softmax_out[Y.flatten(), np.arange(m)] -= 1.0
    dL_dZ = softmax_out
    
    assert(dL_dZ.shape == softmax_out.shape)
    return dL_dZ

def dense_backward(dL_dZ, caches):
    """
    dL_dZ: shape: (vocab_size, m)
    caches: dict. results from each steps of forward propagation
    dL_dZ a des valeurs négatives là où il faut changer qqch. le plus elles sont négatives, le plus c'est grave.
    """
    W = caches['W']
    word_vec = caches['word_vec']
    m = word_vec.shape[1]
    
    # on multiplie là où il faut changer qqch :
    # dL_dZ.shape = (vocab_size, m) 
    dL_dW = (1 / m) * np.dot(dL_dZ, word_vec.T)
    dL_dword_vec = np.dot(W.T, dL_dZ)
    
    assert(W.shape == dL_dW.shape)
    assert(word_vec.shape == dL_dword_vec.shape)
    
    return dL_dW, dL_dword_vec

def backward_propagation(Y, softmax_out, caches):
    dL_dZ = softmax_backward(Y, softmax_out)
    dL_dW, dL_dword_vec = dense_backward(dL_dZ, caches)
    
    gradients = dict()
    gradients['dL_dZ'] = dL_dZ
    gradients['dL_dW'] = dL_dW
    gradients['dL_dword_vec'] = dL_dword_vec
    
    return gradients

def update_parameters(parameters, caches, gradients, learning_rate):
    """
    ici on met le plongement à jour
    """
    vocab_size, emb_size = parameters['WRD_EMB'].shape
    inds = caches['inds']
    WRD_EMB = parameters['WRD_EMB']
    dL_dword_vec = gradients['dL_dword_vec']
    m = inds.shape[-1]
    
    # notez qu'on ne modifie que les lignes correspondants aux mots centraux
    WRD_EMB[inds.flatten(), :] -= dL_dword_vec.T * learning_rate

    parameters['W'] -= learning_rate * gradients['dL_dW']

## Model Training

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt


def skipgram_model_training(X, Y, vocab_size, emb_size, learning_rate, epochs, batch_size=256, parameters=None, print_cost=False, plot_cost=True):
    costs = []
    m = X.shape[1]
        
    if parameters is None:
        parameters = initialize_parameters(vocab_size, emb_size)
    
    # pour l'instant ces trois variables ne sont pas utilisées
    # todo: utiliser ces variables pour garder le meilleur modèle à la fin
    best_epoch = 0
    min_epoch_cost = float('inf')
    parameters['best_embedding'] = parameters['WRD_EMB']
    
    begin_time = datetime.now()
    for epoch in range(epochs):
        epoch_cost = 0
        batch_inds = list(range(0, m, batch_size))
        np.random.shuffle(batch_inds)
        for i in batch_inds:
            X_batch = X[:, i:i+batch_size]
            Y_batch = Y[:, i:i+batch_size]

            softmax_out, caches = forward_propagation(X_batch, parameters)
            cost = cross_entropy(softmax_out, Y_batch)
            gradients = backward_propagation(Y_batch, softmax_out, caches)
            update_parameters(parameters, caches, gradients, learning_rate)
            epoch_cost += cost
            
        costs.append(epoch_cost)
        # todo: ajouter ici le code pour garder le best_embedding
        if epoch_cost<min_epoch_cost:
            parameters['best_embedding'] = parameters['WRD_EMB']
            best_epoch = epoch
            min_epoch_cost = epoch_cost
        # fin todo
        
        if print_cost and epoch % 200 == 0:
            print("Cost after epoch {}: {}".format(epoch, epoch_cost))
        if epoch % (epochs // 100) == 0:
            learning_rate *= 0.98
    end_time = datetime.now()
    print('training time: {}'.format(end_time - begin_time))
    print("TODO I've kept the embedding of epoch {best_epoch} with cost {min_epoch_cost}.".format(best_epoch=best_epoch,min_epoch_cost=min_epoch_cost))        
    if plot_cost:
        plt.plot(np.arange(epochs), costs)
        plt.xlabel('# of epochs')
        plt.ylabel('cost')
    return parameters

In [ ]:
Y.shape, X.shape, Y_one_hot.shape

In [ ]:
Y_one_hot

In [ ]:
parameters = skipgram_model_training(X, Y, VOCAB_SIZE, 50, 0.05, 5000, batch_size=128, parameters=None, print_cost=True)
# tester avec une batch_size petite pour voir....

###### todo:
- garder le meilleur modèle
- visualiser aussi le meilleur modèle